<a href="https://colab.research.google.com/github/baldeaguirre/ollama_rag_agent/blob/main/Implementing_a_RAG_Agent_with_Ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install langchain langchain-community langchain-ollama chromadb pypdf ollama

In [ ]:
import os
import shutil
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain_ollama import OllamaLLM
from langchain.chains import RetrievalQA
import ollama

In [ ]:
# Install and start Ollama in Colab
!curl https://ollama.ai/install.sh | sh
!ollama serve > /dev/null 2>&1 &
!ollama pull llama3.2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13269    0 13269    0     0  64303      0 --:--:-- --:--:-- --:--:-- 64412
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕▏ 7.7 KB                         
pulling a70ff7e

In [ ]:
# Function to process PDF documents
def process_documents(pdf_path):
    # Load PDF
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()

    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_documents(pages)

    return chunks

In [ ]:
# Function to create vector store
def create_vectorstore(chunks):
    # Clean up existing vector store if it exists
    if os.path.exists("chroma_db"):
        shutil.rmtree("chroma_db")

    # Initialize Ollama embeddings
    embeddings = OllamaEmbeddings(model="llama3.2")

    # Create vector store
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory="chroma_db"
    )

    return vectorstore

In [ ]:
# Function to setup RAG pipeline
def setup_rag(vectorstore):
    # Initialize Ollama LLM
    llm = OllamaLLM(model="llama3.2")

    # Create retrieval chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
        return_source_documents=True
    )

    return qa_chain

In [ ]:
# Main RAG query function
def query_rag(qa_chain, query):
    # Get response from the chain
    response = qa_chain.invoke({"query": query})

    return {
        "answer": response["result"],
        "source_documents": response["source_documents"]
    }

In [ ]:
# Example usage
def main():
    # Example PDF path (you would need to upload a PDF to Colab)
    pdf_path = "Short-Term Domestic Energy Consumption Prediction using Machine Learning.pdf"

    # Process documents
    chunks = process_documents(pdf_path)
    print(f"Created {len(chunks)} document chunks")

    # Create vector store
    vectorstore = create_vectorstore(chunks)
    print("Vector store created successfully")

    # Setup RAG pipeline
    qa_chain = setup_rag(vectorstore)
    print("RAG pipeline ready")

    # Example query
    query = "What is the main topic of the document?"
    result = query_rag(qa_chain, query)

    print("\nQuery:", query)
    print("\nAnswer:", result["answer"])

    # print("\nSource Documents:")
    # for i, doc in enumerate(result["source_documents"]):
    #     print(f"\nDocument {i+1}:")
    #     print(doc.page_content[:200] + "...")

if __name__ == "__main__":
    main()

Created 18 document chunks
Vector store created successfully
RAG pipeline ready

Query: What is the main topic of the document?

Answer: Based on the provided context, it appears that the document discusses research related to predicting domestic electrical consumption and balancing energy supply and demand. The main topic seems to be focused on time series prediction using various machine learning models, such as XGBoost, LightGBM, LSTM, GRU, CNN-LSTM, and Seq2Seq with attention, in order to improve the accuracy of short-term predictions and balance between energy supply and demand.
